### Split between train and val image and consequently generate positive and negative pair

In [52]:
import glob
import os
import numpy as np
import random

In [2]:
root_dir = "../lfw/"

In [3]:
all_images = glob.glob(root_dir + "*/*.jpg")

In [4]:
random.shuffle(all_images)

In [5]:
split_factor = 0.8

In [6]:
train_images =  all_images[0:int(len(all_images)*split_factor)]
test_images = all_images[int(len(all_images)*split_factor):]

In [7]:
print(len(train_images),len(test_images))

9528 2382


In [28]:
num_pairs_train = 50000
num_pairs_test = 50000

In [29]:
from tqdm import tqdm 

train_pairs = []

positive_pairs = 0
negative_pairs = 0

for i in tqdm(range(num_pairs_train)):
    img1 = random.choice(train_images)
    class_img1 = img1.split("/")[-2]
    roll_dice = random.randint(0,1)
    
    # get a random number from 0 or 1. This is to ensure that the number of positive and negative pairs are uniformly 
    # distributed
    
    if roll_dice:
        positive_pairs += 1
        while True:
            same_class_images = glob.glob(root_dir + class_img1 + "/*")
            img2 = random.choice(same_class_images)
            if img2 in train_images:
                break
    else:
        negative_pairs +=1 
        while True:
            img2 = random.choice(train_images)
            class_img2 = img2.split("/")[-2]
            
            if(class_img2!=class_img1):
                break  

    img1 = img1.split("../lfw/")[-1]
    img2 = img2.split("../lfw/")[-1]
    
    train_pairs.append([img1,img2,roll_dice]) 

100%|██████████| 50000/50000 [00:23<00:00, 2109.41it/s]


In [30]:
print("Positive pairs : {}".format(positive_pairs))
print("Negative pairs : {}".format(negative_pairs))
print("Ratio of negative to positive pairs : {} ".format((negative_pairs*1.0)/positive_pairs))

Positive pairs : 25036
Negative pairs : 24964
Ratio of negative to positive pairs : 0.9971241412366193 


In [46]:
print("Check how many elements have the same images in the pairs .... ")

same_sample = 0
for pair in train_pairs:
    if pair[0]==pair[1]:
        same_sample = same_sample + 1
    
print(same_sample/50000)

Check how many elements have the same images in the pairs .... 
0.20698


In [33]:
test_pairs = []

for i in tqdm(range(num_pairs_test)):
    img1 = random.choice(test_images)
    class_img1 = img1.split("/")[-2]
    
    # get a random number from 0 or 1. This is to ensure that the number of positive and negative pairs are uniformly 
    # distributed
    
    roll_dice = random.randint(0,1)
    
    if roll_dice:
        while True:
            same_class_images = glob.glob(root_dir + class_img1 + "/*")            
            img2 = random.choice(same_class_images)    
            if img2 in test_images:
                break
    else:
        while True:
            img2 = random.choice(test_images)
            class_img2 = img2.split("/")[-2]
            
            if(class_img2!=class_img1):
                break
                
    img1 = img1.split("../lfw/")[-1]
    img2 = img2.split("../lfw/")[-1]
                
    test_pairs.append([img1,img2,roll_dice])   

100%|██████████| 50000/50000 [00:52<00:00, 952.09it/s] 


In [35]:
print("Check how many elements have the same images in the pairs .... ")

same_sample = 0
for pair in test_pairs:
    if pair[0]==pair[1]:
        same_sample = same_sample + 1
    
print(same_sample/50000)

Check how many elements have the same images in the pairs .... 
0.30628


In [41]:
sample_train = random.sample(train_pairs,3000)
sample_test = random.sample(test_pairs,1000)

In [54]:
np.unique(np.array(sample_train)[:,2],return_counts=True)

(array(['0', '1'], dtype='<U72'), array([1479, 1521]))

In [56]:
np.unique(np.array(sample_test)[:,2],return_counts=True)

(array(['0', '1'], dtype='<U68'), array([514, 486]))

In [12]:
train_writer = open("../files/couples/train.txt","w")
test_writer = open("../files/couples/test.txt","w")

In [13]:
for i in train_pairs:
    train_writer.write("{} {} {}".format(i[0],i[1],i[2]))
    train_writer.write("\n")

In [14]:
for i in test_pairs:
    test_writer.write("{} {} {}".format(i[0],i[1],i[2]))
    test_writer.write("\n")

In [20]:
classes = os.listdir(root_dir)
print("Num Classes : {} ".format(len(classes)))

Num Classes : 4636 


In [24]:
class_mapping = {}

for index,i in enumerate(classes):
     class_mapping[i] = index

In [17]:
train_writer = open("../files/train.txt","w")
test_writer = open("../files/test.txt","w")

In [31]:
for i in train_images:
    i = i.split("../lfw/")[-1]
    class_name = i.split("/")[0]
    class_index = class_mapping[class_name]
    train_writer.write("{} {}".format(i,class_index))
    train_writer.write("\n")
    
for i in test_images:
    i = i.split("../lfw/")[-1]
    class_name = i.split("/")[0]
    class_index = class_mapping[class_name]
    test_writer.write("{} {}".format(i,class_index))
    test_writer.write("\n")
    

In [32]:
class_writer = open("../files/labels.txt","w")

for c in classes:
    class_writer.write("{}".format(c))
    class_writer.write("\n")